### Custom MBA courses RAG project

In [1]:
# Go one level up in the directories hierarchy to access src directory and codes
import sys
import os
# Add project root to Python path
project_root = os.path.abspath("..")  # go one level up from notebooks/
sys.path.append(project_root)

In [2]:
# Setup necessary models for routing, chatting and embedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from core.config.config import Config
from google.genai import types

router_llm = GoogleGenAI(
    model = Config.ROUTER_LLM,
    api_key = Config.GOOGLE_API_KEY,
    generation_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget = 0),
        temperature = Config.ROUTER_LLM_TEMPERATURE,
    ),
    max_tokens = Config.ROUTER_LLM_MAX_TOKENS
)

chat_llm = GoogleGenAI(
    model = Config.CHAT_LLM,
    api_key = Config.GOOGLE_API_KEY,
    generation_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget = 0),
        temperature = Config.CHAT_LLM_TEMPERATURE,
    ),
    max_tokens = Config.CHAT_LLM_MAX_TOKENS
)

embed_model = HuggingFaceEmbedding(
    model_name = Config.EMBEDDING_MODEL
)

c:\Users\Sanzhar\workspace\github.com\kaydarovv\ai_agent_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the collections name and description from my custom JSON file
import json

docs_path = "../documents"
collections_mba_json = docs_path + "/collections_mba.json"

with open(collections_mba_json, "r", encoding = "utf-8") as file:
    COLLECTIONS_MBA = json.load(file)

for collections_name, collection_description in COLLECTIONS_MBA.items():
    COLLECTIONS_MBA[collections_name] = (" \n ").join([line.strip() for line in collection_description.splitlines()[1:-2]])

In [4]:
# Let's test our RAG module defined in class to see how well it is refactored
import nest_asyncio
from core.src.rag_workflow import RagSystem
nest_asyncio.apply()

rag_instance = RagSystem(
    router_llm = router_llm,
    chat_llm = chat_llm,
    embed_model = embed_model,
    docs_path = docs_path,
    collections = COLLECTIONS_MBA
)

2025-12-15 10:46:46,784 - WARNING - Ignoring wrong pointing object 9 0 (offset 0)
2025-12-15 10:46:46,785 - WARNING - Ignoring wrong pointing object 20 0 (offset 0)
2025-12-15 10:46:46,786 - WARNING - Ignoring wrong pointing object 22 0 (offset 0)
2025-12-15 10:46:46,786 - WARNING - Ignoring wrong pointing object 30 0 (offset 0)
2025-12-15 10:46:46,787 - WARNING - Ignoring wrong pointing object 36 0 (offset 0)
2025-12-15 10:46:46,787 - WARNING - Ignoring wrong pointing object 49 0 (offset 0)
2025-12-15 10:46:46,788 - WARNING - Ignoring wrong pointing object 55 0 (offset 0)
2025-12-15 10:46:46,789 - WARNING - Ignoring wrong pointing object 82 0 (offset 0)
2025-12-15 10:46:46,789 - WARNING - Ignoring wrong pointing object 84 0 (offset 0)
2025-12-15 10:46:46,790 - WARNING - Ignoring wrong pointing object 95 0 (offset 0)
2025-12-15 10:46:46,790 - WARNING - Ignoring wrong pointing object 112 0 (offset 0)
2025-12-15 10:46:46,791 - WARNING - Ignoring wrong pointing object 115 0 (offset 0)
202

In [10]:
# Let's test the query and memory of the RAG system
user_name = "SomeNewUSer"
user_id = "test_user_id0"
user_query = "What can you say about behaviour theories used in Marketing?"

rag_response = await rag_instance.query(user_name, user_id, user_query)
print(rag_response)

2025-12-15 10:52:46,703 - INFO - AFC is enabled with max remote calls: 10.
2025-12-15 10:52:47,724 - INFO - Selecting retriever 3: This choice explicitly mentions 'consumer behavior' and states that the course will delve into it by 'applying behavioral theories to understand how individuals make consumption decisions.' It further elaborates on how 'internal psychological factors' and 'external influences' shape these decisions and how this study helps explain consumer choices and purchase decisions..
2025-12-15 10:52:47,750 - INFO - AFC is enabled with max remote calls: 10.
2025-12-15 10:52:48 | INFO     | core.src.rag_workflow:_get_or_create_chat_engine:158 - Creating a new chat engine for user {user_id}
2025-12-15 10:52:48,847 - INFO - AFC is enabled with max remote calls: 10.


SomeNewUSer,

Based on the provided materials, behavior theories in Marketing are primarily discussed in the context of learning. These theories posit that conditioning results in learning. Specifically, behavioral learning theories concentrate on stimulus-response connections, where learning occurs through observable behaviors (responses) triggered by external stimuli. Two key types of behavioral learning theories mentioned are:

*   **Classical conditioning**
*   **Instrumental Conditioning**

Additionally, the materials briefly touch upon Freudian principles in relation to consumer behavior, suggesting that advertisements can portray aspects of these principles.


In [11]:
# Let's test the query and memory of the RAG system
user_query = "What else can you say?"

rag_response = await rag_instance.query(user_name, user_id, user_query)
print(rag_response)

2025-12-15 10:53:36,790 - INFO - AFC is enabled with max remote calls: 10.
2025-12-15 10:53:38,674 - INFO - Selecting retriever 5: This choice discusses the integration of different perspectives learned in other courses and applying them to complex management situations, which aligns with the open-ended nature of the question 'What else can you say?' It emphasizes holistic thinking and developing judgment as a general manager..
2025-12-15 10:53:38,675 - INFO - Selecting retriever 2: This choice focuses on providing an overview of management and managing organizations, introducing core elements, and developing analytical skills for decision-making. This broad scope allows for a wide range of discussions and insights, fitting the 'What else can you say?' prompt..
2025-12-15 10:53:38,676 - INFO - Selecting retriever 3: This choice delves into strategic marketing and consumer behavior, emphasizing decision-making in a changing environment and applying behavioral theories. The complexity an

SomeNewUSer,

The provided context also mentions that we learn about products by observing others' behavior, which falls under Cognitive Theories. Cognitive theories, in contrast to behavioral learning theories, view consumers as rational problem solvers who learn by observing relationships and through mental information processing in response to problem-solving.
